In [ ]:
import jax

regression model (X 와 Y간의 선형적 관계계)

In [ ]:
#활성화 함수 정이ㅡ
def nonlin(x):
    return jax.nn.tanh(x)

In [ ]:
#BNN Model
# a two layer bayesian neural network witn computational flow
#given by D_X => D_H => D_Y where D_Y is the number of
#gidden units , {note we indicate tensor dimesions in the comments}

def model(X,Y ,D_H ,D_Y =1):
    N, D_X = X ,shape

    #samlple first layer {we put unit normal priors on all weights}
    w1 =numpyro.sample("w1", dist.Normal(jnp.zeros((D_X, D_H)),
                                                jnp.ones((D_X, D_H))))
    assert w1.shape == (D_X, D_H)
    z1 = nonlin(jnp.matmul(X, w1)) # first layer of activators
    assert z1.shape == (N, D_H)

    #sample second layer
    w2 = numpyro.sample("w2", dist.Normal(jnp.zeros((D_H, D_H)),
                                                jnp.ones((D_H, D_H))))
    assert w2.shape == (D_H, D_H)
    z2 = nonlin(jnp.matmul(z1, w2)) #second layer of activators
    assert z2.shape == (N, D_H)

    #sample final layer of weights and neural network output
    w3 = numpyro.sample("w3", dist.Normal(jnp.zeros((D_H, D_Y)),
                                                jnp.ones((D_H, D_Y))))
    assert w3.shape == (D_H, D_Y)
    z3 = jnp.matmul(z2, w3)  #output of the neural network
    assert z3.shape == (N, D_Y)

    if Y is not None:
        assert z3.shape == Y.shape

    #we  put a prior in the observation noise
    prec_obs = numpyro.sample("prec_obs", dist.Gamma(3.0, 1.0))
    sigma_obs = 1.0/ jnp.sqrt(prec_obs)

    #observe date
    with numpyro.plate("data", N):
        #note we use to_event(1) because each obsercation has shape(1,)
        numpyro.sample("Y", dist.Normal(z3, sigma_obs).to_event(1), obs=Y)



In [ ]:
#변분분포를 지정해야함 , 정규분포 활용하면 빠르게 할 수 있음(파라미터마다 따로 지정 안하고..)
from numpyro.infer.autoguide import AutoMultivariateNormal
from numpyro.infer.autoguide import AutoDiagonalNormal

guide = AutoMultivariateNormal(model)
#guide = AutoDiagonalNormal(model)
#가이드와 모델을 가지고 SVI 객체 만들어 구현해보기